<a href="https://colab.research.google.com/github/Kilonzus/dm-class/blob/master/fin-report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cover Section

**Report done by Peter Kilonzo**

This report details approaches that I took along with a group member for the final projects of a Data Mining class.

**Note** sometimes google colab will throw an error when trying to upload files. I find that by rerunning the cell this error disappears.

## List of Documents

### Task 1

1.   label_training.txt - list of labels corresponding to data in training.txt
2.   training.txt - training data, respresents sparse matrix
3.   test_sample.xlsx - used to test our data



### Task 2

1.   hr_rad_temp_pr_hu_drt-10.csv - csv file containing two weeks of data from Sheep 1, sheep 2, and sheep 3, and sheep 4. Data from sheep 1 is Temperature, Atmospheric Pressure, humidity, Dirt Temperature, Solar Radiation. Data from sheep 2, 3, and 4 is Solar Radiation. It appears in the order listed.



### Task 3



1.   transformed2.xlsx - excel file containng data provided by Market Evolution



# **Task 1**

Our goal for task one was to build a classification model to predict the label for given data. 

## Data Preprocessing

Originally we thought that the data provided was just a huge amount of data for us to train with. Upon closer inspection it was realized the data in the provided Training.txt was actually a representation of a sparse matrix.We will first pull in our data then we will examine it.

In [0]:
import pandas as pd
import numpy as np
from google.colab import files
import io

#uploaded = files.upload()
np.set_printoptions(suppress=True)
df1 = pd.read_csv(io.BytesIO(uploaded["training.txt"]),sep="\s+",header=None)
df1.columns=['ID','Features','Values']


Saving training.txt to training (3).txt


In [0]:
df1

,ID,Features,Values
0,1,6,1.00
1,1,160,31.00
2,1,438,1.00
3,1,479,1.00
4,1,618,1.00
...,...,...,...
443398,1842,25905,65.00
443399,1842,25930,34.00
443400,1842,26083,0.78
443401,1842,26181,0.52


As we can see the data originally appears as a 443403 x 3 matrix.

In [0]:
#uploaded2 = files.upload()
#uploaded3 = files.upload()
df3 = pd.read_excel(io.BytesIO(uploaded2["test_sample.xlsx"]),sheet_name="data",sep="\s+",header=None)
df3.columns=['ID','Features','Values']
mat1 = pd.pivot_table(df1, index='ID', columns='Features', values='Values')
mat1 = mat1.fillna(0)
mat3 = pd.pivot_table(df3, index='ID', columns='Features', values='Values')
mat3 = mat3.fillna(0)
df2 = pd.read_csv(io.BytesIO(uploaded3["label_training.txt"]),sep="\t",header=None)
df2.columns = ['Classes']
df4 = pd.read_excel(io.BytesIO(uploaded2["test_sample.xlsx"]),sheet_name="label",sep="\t",header=None)
df4.columns = ['Classes']

In [0]:
mat1

Features,2,3,4,5,6,8,10,12,15,21,22,23,25,26,27,28,29,31,33,36,38,39,41,42,44,51,52,53,56,63,64,65,66,72,76,78,80,83,85,89,...,26299,26300,26302,26304,26305,26308,26309,26310,26312,26313,26315,26316,26318,26319,26320,26321,26322,26326,26327,26332,26334,26335,26336,26337,26338,26339,26340,26344,26348,26349,26350,26351,26352,26354,26355,26356,26357,26360,26362,26364
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


After converting it into its proper form we see that it is very sparse with most values of the matrix being 0.

We use the truncated single value decompsition technique to decrease the dimensionality of our data.

In [0]:
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
mat1 = svd.fit_transform(mat1)

Now that we have formatted our data appropriately, we can split it into training and testing data

In [0]:
xtrain, xtest = train_test_split(mat1, test_size=0.2, shuffle=False)
ytrain, ytest = train_test_split(df2, test_size=0.2, shuffle=False)
ytrain = np.ravel(ytrain)
ytest = np.ravel(ytest)

## Model Selection and fitting

We used a support vector machine for this problem as that gave us the best performance and also is pretty transparent. 

In [0]:
from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.SVC(kernel='linear')
clf.fit(xtrain, ytrain)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

## Model Testing

Now we test the accuracy of our model

In [0]:
pred = clf.predict(xtest)
accuracy_score(ytest,pred)

0.962059620596206

## Reflections

While this task was the first one it was perhaps the most difficult. At first we thought we could just use a decision tree on the original data by just appending the label to the end of each value and using that to train/test our model. It wasn't until we were running into bad results that we realized that the data probably was not as simple as we thought it was. Upon realizing that it was very sparse data we still had to deal with what methods would work best for us to reduce some of the dimensions. We also had to deal with a greater amount of attributes vs features. Our methods did pay off with a high accuracy for model.

# **Task 2**


For task 2, our goal was to predict solar radiation level at the site Mojave Desert Shrub. Specifically, we wanted to predict the solar radiatin levels of Mojave Desert Shrub on Labor Day 2019. We had three approaches to this program, for our first approach we used just parameters made from data at our target site. To explain further, we used data such as temperature, air pressure, soil temperature, and time to predict the solar radiation levels. I explore our first approach below.

## Data Preprocessing

Firstly we need our data. Credit goes to the NDRC (Nevada Desert Research Center) for providing the data. After downloading our data we first examined the data. We noticed that the first ~8 rows were filled with information about the data that we really did not need.![alt text](https://drive.google.com/uc?id=1vo_IjB3gaBPcTL8fWNX1Z16bIjVCwqVD)

As shown the first 9 lines of our data was not really pertinant to our task as long as we understand what it is. We also did not need the 'Number of Contituent Values' columns so we used spreadsheet software (Libre Office Calc and Microsoft Excel) to delete thesee columns and rows. We also used the average values and discarded the other statistical data such as standard deviation, maximum, and mininum. Thus our data appears as so ![alt text](https://drive.google.com/uc?id=1nAVL-cneNXBU5tcO0_SrQrEP8PnHkNRI)

This data is times series data. Linear Regression does not usually take this into account. To deal with this we changed that column to only be the hour measured in 'military time' (24 hour clock model with 0 indicating midnight and 12 indicating noon). Below we import the data and do this using Python Code.

In [0]:
import pandas as pd
import numpy as np
from google.colab import files
import io

uploaded = files.upload()
data_names = ['Hour', 'Temperature', 'Humidity', 'Atmospheric Pressure', 'Soil Temperature', 'Solar Radiation', 'solar_rads_s2', 'solar_rads_s3', 'solar_rads_s4'] #name of each column
data = pd.read_csv(io.BytesIO(uploaded['hr_rad_temp_pr_hu_drt-10.csv']), names = data_names)

x = data[['Hour', 'Temperature', 'Atmospheric Pressure', 'Soil Temperature']].values #sets the parameters for our linear model i.e. the values we will use to predict solar radiation.

y = data['Solar Radiation'].to_numpy().reshape(-1, 1) #this is our dependent variable i.e. the value we predict

for i in range(len(x)): #this changes each timestamp entry into just the hour values in integer value
	x[i][0] = int(x[i][0][11]+x[i][0][12])

Saving hr_rad_temp_pr_hu_drt-10.csv to hr_rad_temp_pr_hu_drt-10 (6).csv


As we can see x now has a numeric value indicating our for the first value.

In [0]:
x

array([[18, 36.39, 905.1975, 41.59],
       [18, 35.49, 905.2341, 41.29],
       [19, 35.26, 905.2462, 40.99],
       ...,
       [18, 37.25, 908.9097, 41.03],
       [18, 37.18, 908.9176, 40.81],
       [18, 36.78, 908.9687, 40.59]], dtype=object)

Next we upload our testing data to be used later

In [0]:
uploaded = files.upload()
f_testn = ['Hour', 'Temperature', 'Humidity', 'Atmospheric Pressure', 'Soil Temperature', 'Solar Radiation', 'solar_rads_s2', 'solar_rads_s3', 'solar_rads_s4']
f_testdata = pd.read_csv(io.BytesIO(uploaded["f-fin.csv"]), names = f_testn)

fx = f_testdata[['Hour', 'Temperature', 'Atmospheric Pressure', 'Soil Temperature']].values
fy = f_testdata['Solar Radiation'].to_numpy().reshape(-1, 1)

for i in range(len(fx)):
	fx[i][0] = int(fx[i][0][11]+fx[i][0][12])

Saving f-fin.csv to f-fin (7).csv


## Approach 1

### Model Selection and fitting

Now we split our data in to training and testing data, we use 20% of our data for testing and 80% for training. We then fit our model

In [0]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

regr = linear_model.LinearRegression()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=0)
regr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Model Testing

Now we test our model and see what our RMSE and r^2 values are for our training set.

In [0]:
y_pred_test = regr.predict(x_test)
print("Root mean squared error = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred_test)))
print('MSE = %.4f' % mean_absolute_error(y_test, y_pred_test))

Root mean squared error = 0.3528
MSE = 0.2861


Now to test it aganist our testing set and see the RMSE and r^2 values:

In [0]:

f_pred_test = regr.predict(fx)
print("Results for labor day prediction :\n")
print("Root mean squared error = %.4f" % np.sqrt(mean_squared_error(fy, f_pred_test)))
print('MSE = %.4f' % mean_absolute_error(fy, f_pred_test))


Results for labor day prediction :

Root mean squared error = 0.6866
MSE = 0.6102


While small RSME values may seem like a good thing, we have to realize that our data values are already low. Solar Radiation rarely exceeds 1.1 thus we want to see whether we can lower this even further.

## Approach 2

### Model Selection and fitting

This time we use radiation monitoring from the other Sheep Range sites to tune our Linear Model. This is included in our Data from before so no extra data is needed.

In [0]:
x = data[['Hour', 'Soil Temperature', 'solar_rads_s4']].values

for i in range(len(x)):
	x[i][0] = int(x[i][0][11]+x[i][0][12])

y = data['Solar Radiation'].to_numpy().reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

fx = f_testdata[['Hour', 'Soil Temperature', 'solar_rads_s4']].values
fy = f_testdata['Solar Radiation'].to_numpy().reshape(-1, 1)

for i in range(len(fx)):
	fx[i][0] = int(fx[i][0][11]+fx[i][0][12])

regr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Model Testing

Now we test our model

In [0]:
y_pred_test = regr.predict(x_test)
print("Root mean squared error = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred_test)))
print('MSE = %.4f' % mean_absolute_error(y_test, y_pred_test))

Root mean squared error = 0.2320
MSE = 0.1259


In [0]:

f_pred_test = regr.predict(fx)
print("Results for labor day prediction :\n")
print("Root mean squared error = %.4f" % np.sqrt(mean_squared_error(fy, f_pred_test)))
print('MSE = %.4f' % mean_absolute_error(fy, f_pred_test))


Results for labor day prediction :

Root mean squared error = 0.1637
MSE = 0.1033


By choosing our parameters carefully we see we can cut our error to about 24% of the value we had before. It should be noted that when we use data from each site we end up overfitting with our training rsme being miniscule (about 0.023) and our testing rsme being high relative to our testing value (about 0.204).

## Reflections

One important thing perhaps in all data mining tasks is to have some understanding of the data that you are performing your data mining techniques on. In this example it was important to understand what solar radiation is as well as what factors can affect it. At night the sun is not our so obviously there would be no solar radiation. The earth rotates about its own axis and follows a non circular route around the sun meaning radiation is peak at mid day and that radiation levels are higher in January vs July. We can see this effect in our data as even from August 21st to September 2nd there was a shift in the time of day that radiation started and stopped. We aimed for a model that we could tailor our data too as well as understand clearly what it was doing. A linear model offered us great performance as well as high transparancy. 

# **Task 3**

For task 3 our goal was to predict the vehicle car type based parameters we selected from the provided dataset. Credit goes to Market Evolution for providing the dataset used for this task.

## Data Preprocessing

One of the more challenging parts about this task is the dataset. It contains many missing values and incomplete data. Not to mention it also has a great amount of features which we may not need in constructing our model. To start our data preprocessing we removed all features that were missing more than 5166 values. We settle upon the number 5166 after probing our dataset. With 280 original features removing 5166 left us with 205 remaining features for us to use. We found increasing this value even by one severely reduced the amount of features.

First pull in the data and drop a column using pandas

In [0]:
import pandas as pd
from google.colab import files
import io
import numpy as np

uploaded = files.upload()
df = pd.read_excel(io.BytesIO(uploaded["transformed2.xlsx"]))
df.drop(['Unnamed: 0'],axis=1,inplace=True)

Saving transformed2.xlsx to transformed2 (1).xlsx


Next we fill the empty values with averages and the fill forward technique using pandas. We also split our data as well as make sure it is in an appropriate form for our model (i.e. set y to person type, drop person type from x, and create a list of our time data and categorical data). We also use label encoder to give our string values ordinal numeric values.

In [0]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

df = df.fillna(df.mean())
df1 = df.fillna(method='ffill')
ysplit = df1['Person Type']
xsplit = df1.drop(['Person Type'],axis=1)
dt = xsplit.select_dtypes(include=['datetime64'])
cat2 = list(dt.columns)
categorical = xsplit.select_dtypes(exclude=['float64'])
cat3 = list(categorical.columns)
cols = cat2 + cat3
le = preprocessing.LabelEncoder()
xsplit[cols] = xsplit[cols].apply(lambda col: le.fit_transform(col))


Now since we have so much data we select the best features of our data using a chisquared distribution

In [0]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(xsplit,ysplit)

Finally we can create a new training set which essential features and numeric values for all of them.

In [0]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(xsplit.columns)
fes = pd.concat([dfcolumns,dfscores],axis=1)
fes.columns = ['Specs','Score']
fes.nlargest(15,'Score')
fes = fes.sort_values(by =['Score'],ascending=False)
large = fes.head(10)

corr = large['Specs'].values
xsplit1 = xsplit[corr] 

## Model Selection and fitting

Now that we have a good set of training and testing data we can fit our data. We decided to use a decision tree for this task. As it gave us the best results out of all the other classification algorithms we tried. Decision Trees also offer a good amount of transparency.

In [0]:
from sklearn import tree

xtrain, xtest = train_test_split(xsplit1, test_size=0.2, shuffle=False)
ytrain, ytest = train_test_split(ysplit, test_size=0.2, shuffle=False)

clf = tree.DecisionTreeClassifier()
clf.fit(xtrain, ytrain)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
xtest

,Census 2010: Tract and block group,Estimated Current Home Value,CAPE: HomVal: OOHU: Median Home Value,CAPE: Inc: HH: Median Family Household Income,Acct#,Person Combined Age,USNG,Estimated Monthly Mortgage Pay - Amount,Person Marital Status,Household Composition
7999,2.660858e+06,184806.215641,201428.006194,73758.257453,3141,46,2875,591.786295,1,6
8000,2.660858e+06,184806.215641,201428.006194,73758.257453,3278,46,2875,591.786295,1,6
8001,2.660858e+06,184806.215641,201428.006194,73758.257453,4223,46,2875,591.786295,1,6
8002,2.660858e+06,184806.215641,201428.006194,73758.257453,5948,46,2875,591.786295,1,6
8003,3.653022e+06,120143.000000,147114.000000,59582.000000,6256,24,2956,0.000000,1,8
...,...,...,...,...,...,...,...,...,...,...
9994,2.660858e+06,184806.215641,201428.006194,73758.257453,974,14,542,591.786295,0,3
9995,2.660858e+06,184806.215641,201428.006194,73758.257453,572,14,542,591.786295,0,3
9996,1.290210e+05,125357.000000,156480.000000,60483.000000,4011,21,218,789.000000,1,2
9997,2.660858e+06,184806.215641,201428.006194,73758.257453,6066,21,218,591.786295,1,2


Testing our model

In [0]:
ypred = clf.predict(xtest)
accuracy_score(ytest, ypred)

0.9765

As seen we get a pretty high accuracy score.

## Reflections

This project was one of the more difficult ones for us to complete however it turned out well. The most pressing thing about this task was the incomplete nature of the data. A lot of time went into figuring out just how we should deal with data so incomplete. This task was heavy on data preprocessing techniques. A worry with this approach is our selection of best features. As you can see above, we used a chi-squared test to pick our features. We see that it chose Tract and block group as well as USNG information. Both of these are location information. It also choose account number. Originally I thought these were terrible choices but our model had the highest accuracy score with these. Another was the complexity of the actual decision tree. I have included the code below as well as including a PDF in my submission of the output. As seen it can get quite deep and be troublesome for a human to follow along.

In [0]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("Car")